In [2]:
import pandas as pd
import numpy as np


In [4]:
df_train = pd.read_csv("GUIDE_Train.csv", nrows=10000)
df_test = pd.read_csv("GUIDE_Test.csv", nrows=10000)

In [5]:
df_train.head(5)

,Id,OrgId,IncidentId,AlertId,Timestamp,DetectorId,AlertTitle,Category,MitreTechniques,IncidentGrade,...,ResourceType,Roles,OSFamily,OSVersion,AntispamDirection,SuspicionLevel,LastVerdict,CountryCode,State,City
0,180388628218,0,612,123247,2024-06-04T06:05:15.000Z,7,6,InitialAccess,NaN,TruePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,31,6,3
1,455266534868,88,326,210035,2024-06-14T03:01:25.000Z,58,43,Exfiltration,NaN,FalsePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630
2,1056561957389,809,58352,712507,2024-06-13T04:52:55.000Z,423,298,InitialAccess,T1189,FalsePositive,...,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
3,1279900258736,92,32992,774301,2024-06-10T16:39:36.000Z,2,2,CommandAndControl,NaN,BenignPositive,...,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
4,214748368522,148,4359,188041,2024-06-15T01:08:07.000Z,9,74,Execution,NaN,TruePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 45 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  10000 non-null  int64  
 1   OrgId               10000 non-null  int64  
 2   IncidentId          10000 non-null  int64  
 3   AlertId             10000 non-null  int64  
 4   Timestamp           10000 non-null  object 
 5   DetectorId          10000 non-null  int64  
 6   AlertTitle          10000 non-null  int64  
 7   Category            10000 non-null  object 
 8   MitreTechniques     4201 non-null   object 
 9   IncidentGrade       9947 non-null   object 
 10  ActionGrouped       59 non-null     object 
 11  ActionGranular      59 non-null     object 
 12  EntityType          10000 non-null  object 
 13  EvidenceRole        10000 non-null  object 
 14  DeviceId            10000 non-null  int64  
 15  Sha256              10000 non-null  int64  
 16  IpAdd

In [7]:
missing_values = df_train.isnull().sum()
missing_percentages = 100 * missing_values / len(df_train)

# Display columns with missing values
missing_data = pd.concat([missing_values, missing_percentages], axis=1, keys=['Total', 'Percent'])
print(missing_data[missing_data['Total'] > 0].sort_values('Percent', ascending=False))


                   Total  Percent
ResourceType        9992    99.92
ActionGranular      9941    99.41
ActionGrouped       9941    99.41
ThreatFamily        9923    99.23
EmailClusterId      9881    98.81
AntispamDirection   9823    98.23
Roles               9789    97.89
SuspicionLevel      8462    84.62
LastVerdict         7600    76.00
MitreTechniques     5799    57.99
IncidentGrade         53     0.53


In [11]:
df_train_drop = df_train.drop(columns = ["ResourceType","ActionGranular","ActionGrouped","ThreatFamily","EmailClusterId","AntispamDirection","Roles"])

In [13]:
df_train_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Id                  10000 non-null  int64 
 1   OrgId               10000 non-null  int64 
 2   IncidentId          10000 non-null  int64 
 3   AlertId             10000 non-null  int64 
 4   Timestamp           10000 non-null  object
 5   DetectorId          10000 non-null  int64 
 6   AlertTitle          10000 non-null  int64 
 7   Category            10000 non-null  object
 8   MitreTechniques     4201 non-null   object
 9   IncidentGrade       9947 non-null   object
 10  EntityType          10000 non-null  object
 11  EvidenceRole        10000 non-null  object
 12  DeviceId            10000 non-null  int64 
 13  Sha256              10000 non-null  int64 
 14  IpAddress           10000 non-null  int64 
 15  Url                 10000 non-null  int64 
 16  AccountSid          100

In [14]:
le_cat_columns = ['Category', 'EntityType', 'EvidenceRole', 'SuspicionLevel', 'LastVerdict',
                  'CountryCode', 'OSFamily', 'OSVersion','State', 'City', 'RegistryValueName', 'RegistryValueData', 
                  'ResourceIdName', 'RegistryKey', 'OAuthApplicationId', 'ApplicationId', 'ApplicationName']

numerical_columns = ['DeviceId', 'Sha256', 'IpAddress', 'Url', 'AccountSid', 'AccountUpn', 'AccountObjectId',
                     'AccountName', 'DeviceName', 'NetworkMessageId', 'FileName', 'FolderPath']

le_cat_columns += numerical_columns

numerical_columns = []

ohe_cat_columns = []

In [15]:
df_train_drop[le_cat_columns].nunique().sort_values(ascending=False)

AccountUpn            3386
AccountName           2323
AccountSid            2211
AccountObjectId       2200
IpAddress             1879
NetworkMessageId      1193
FileName               706
DeviceName             686
Url                    603
Sha256                 512
FolderPath             465
DeviceId               375
City                   196
State                  136
CountryCode             68
ApplicationName         23
ApplicationId           21
EntityType              20
Category                17
ResourceIdName          12
RegistryKey             10
OSVersion                6
RegistryValueName        5
RegistryValueData        4
LastVerdict              3
OSFamily                 3
OAuthApplicationId       3
EvidenceRole             2
SuspicionLevel           2
dtype: int64